บทเรียนนี้จะเน้นการจัดการข้อมูลหลายชุด (Multiple Data Sources) ซึ่งเป็นงานจริงที่ป๋าต้องเจอแน่นอน เช่น การเอารายงานหลายเดือนมารวมกัน หรือการดึงชื่อสินค้าจากตาราง Master มาแปะ

In [2]:
import pandas as pd
import numpy as np
import glob


In [3]:

# --- 1. สร้างไพล์ยอดขายรายเดือน (สำหรับโจทย์รวมไฟล์) ---
# เดือนมกราคม
df_jan = pd.DataFrame({
    'Order_ID': ['J001', 'J002', 'J003'],
    'Product_Code': ['P1', 'P2', 'P1'],
    'Qty': [10, 5, 20]
})
df_jan.to_csv('../datasets/multiple_data/sales_jan.csv', index=False)

# เดือนกุมภาพันธ์
df_feb = pd.DataFrame({
    'Order_ID': ['F001', 'F002'],
    'Product_Code': ['P3', 'P1'],
    'Qty': [8, 15]
})
df_feb.to_csv("../datasets/multiple_data/sales_feb.csv", index=False)

# --- 2. สร้างไฟล์ Master สินค้า (สำหรับโจทย์ VLOOKUP) ---
df_master = pd.DataFrame({
    'P_Code': ['P1', 'P2', 'P3', 'P4'],
    'P_Name': ['ปากกา', 'ดินสอ', 'ยางลบ', 'ไม้บรรทัด'],
    'Price': [10, 5, 3, 12]
})
df_master.to_csv('../datasets/multiple_data/product_master.csv', index=False)

# --- 3. สร้างไฟล์เรทราคาน้ำหนัก (สำหรับโจทย์ XLOOKUP - Approximate Match) ---
# น้ำหนักไม่เกินนี้ คิดราคานี้ (Tier ราคา)
df_rates = pd.DataFrame({
    'Max_Weight': [1, 5, 10, 20],
    'Shipping_Cost': [30, 50, 80, 150]
})
df_rates.to_csv('../datasets/multiple_data/shipping_Cost.csv', index=False)

print('สร้างไฟล์จำลองเสร็จเรียบร้อย')

สร้างไฟล์จำลองเสร็จเรียบร้อย


1. รวมไฟล์ Excel/CSV หลายไฟล์เป็นอันเดียว (Concatenation)
โจทย์: ฝ่ายขายส่งไฟล์ยอดขายแยกเดือนมา ป๋าต้องรวมเป็นก้อนเดียวเพื่อทำกราฟปี



แบบง่าย (Easy Mode)
ถ้ารู้ชื่อไฟล์แน่นอน และมีแค่ไม่กี่ไฟล์ จับมาใส่ List แล้วรวมเลยครับ

In [4]:
df1 = pd.read_csv('../datasets/multiple_data/sales_jan.csv')
df2 = pd.read_csv('../datasets/multiple_data/sales_feb.csv')

# รวมร่างด้วย pd.concat (เอามาต่อท้าย)
all_sales = pd.concat([df1, df2], ignore_index=True)

print(df1, '\n')
print(df2, '\n')
print(all_sales)

  Order_ID Product_Code  Qty
0     J001           P1   10
1     J002           P2    5
2     J003           P1   20 

  Order_ID Product_Code  Qty
0     F001           P3    8
1     F002           P1   15 

  Order_ID Product_Code  Qty
0     J001           P1   10
1     J002           P2    5
2     J003           P1   20
3     F001           P3    8
4     F002           P1   15


แบบยาก (Pro Mode: Automate)
สถานการณ์จริง: ถ้าป๋ามีไฟล์ 12 เดือน หรือ 100 ไฟล์ จะมานั่งพิมพ์ชื่อไฟล์ทีละบรรทัดไม่ไหว และป๋าต้องการรู้ด้วยว่า "ข้อมูลนี้มาจากไฟล์ไหน?" (Source Identification)

เราจะใช้ไลบรารี glob มาช่วยกวาดชื่อไฟล์ทั้งหมดในโฟลเดอร์ครับ

In [5]:
# 1. หาไฟล์ทั้งหมดที่ขึ้นต้นด้วย sales_ และลงท้ายด้วย .csv

file_list = glob.glob('../datasets/multiple_data/sales_*.csv')

all_data = [] # สร้าง list ว่างไว้รอเก็บ
for filename in file_list:
    df = pd.read_csv(filename)
    df['Source_File'] = filename        # เทคนิคสำคัญ * สร้างคอลัมน์บอกที่มา
    all_data.append(df)

# รวมครั้งเดียว
final_df = pd.concat(all_data, ignore_index=True)

print('--- รวมแบบ Pro (รู้ที่มา) ---')
final_df.to_excel('../output/concat_sales.xlsx')
print(final_df)

--- รวมแบบ Pro (รู้ที่มา) ---
  Order_ID Product_Code  Qty                              Source_File
0     F001           P3    8  ../datasets/multiple_data\sales_feb.csv
1     F002           P1   15  ../datasets/multiple_data\sales_feb.csv
2     J001           P1   10  ../datasets/multiple_data\sales_jan.csv
3     J002           P2    5  ../datasets/multiple_data\sales_jan.csv
4     J003           P1   20  ../datasets/multiple_data\sales_jan.csv


2. จำลอง VLOOKUP (การดึงข้อมูลข้ามตาราง)
โจทย์: ในตารางยอดขายมีแค่รหัสสินค้า (Product_Code) ป๋าอยากได้ชื่อสินค้าและราคาจากไฟล์ Master มาแปะ

แบบง่าย (Easy Mode: ชื่อคอลัมน์เหมือนกัน)
ใน Pandas เราใช้คำสั่ง merge ครับ (SQL เรียกว่า Join)

สมมติชื่อคอลัมน์ตรงกันเป๊ะๆ มันจะง่ายมาก

In [6]:
# โค๊ดจำลอง กรณีเปลี่ยนชื่อให้เหมือนกันก่อน)
sales = final_df.copy()
master = pd.read_csv('../datasets/multiple_data/product_master.csv')

# VLOOKUP แบบ Excel Match (how='left' คือยึดตารางขายเป็นหลัก)
# หมายเหตุ: ในไฟล์ตัวอย่าง ชื่อคอลัมน์ไม่ตรงกัน (Product_Code vs P_Code)
# ถ้าเหมือนกันจะใช้ on='Product_Code' ได้เลย
master


,P_Code,P_Name,Price
0,P1,ปากกา,10
1,P2,ดินสอ,5
2,P3,ยางลบ,3
3,P4,ไม้บรรทัด,12


แบบยาก (Real World Mode: ชื่อคอลัมน์ไม่เหมือน + Keys ซับซ้อน)
สถานการณ์จริง: ตารางยอดขายใช้ชื่อ Product_Code แต่ตาราง Master ใช้ชื่อ P_Code (คนละชื่อกัน) และเราอยากดึงข้อมูลมาแค่บางส่วน

In [7]:
master = pd.read_csv('../datasets/multiple_data/product_master.csv')

# ใช้ left_on และ right_on เพื่อบอกคู่ที่จับคู่กัน
merged_df = pd.merge(
    sales,
    master[['P_Code', 'P_Name', 'Price']],
    left_on='Product_Code',
    right_on='P_Code',
    how='left'
)

# ลบคอลัมน์ที่เกินมาทึ้ง (P_Code จะติดมาด้วย หลังจาก merged)
merged_df = merged_df.drop(columns=['P_Code','Source_File'])

print('--- ผลลัพธ์ VLOOKUP (Merge) ---')
merged_df

--- ผลลัพธ์ VLOOKUP (Merge) ---


,Order_ID,Product_Code,Qty,P_Name,Price
0,F001,P3,8,ยางลบ,3
1,F002,P1,15,ปากกา,10
2,J001,P1,10,ปากกา,10
3,J002,P2,5,ดินสอ,5
4,J003,P1,20,ปากกา,10


3. จำลอง XLOOKUP (การค้นหาแบบยืดหยุ่น / Approximate Match)
XLOOKUP ใน Excel เก่งกว่า VLOOKUP ตรงที่มันหาค่า "ใกล้เคียงที่สุด" ได้ (เช่น การตัดเกรด หรือคิดค่าส่งตามช่วงน้ำหนัก)

แบบง่าย (Easy Mode: XLOOKUP หาค่า + จัดการค่าว่าง)
XLOOKUP มักใช้จัดการกรณี "หาไม่เจอ" ให้ใส่ค่า Default ใน Pandas เราทำหลัง Merge เสร็จครับ

In [9]:
# สมมติมียอดขายสินค้า P99 (ไม่มีใน Master)
# พอ Merge เสร็จ ค่าจะเป็น NaN (Not a Number)
merged_df['P_Name'] = merged_df['P_Name'].fillna('ไม่ระบบชื่อสินค้า')
merged_df


,Order_ID,Product_Code,Qty,P_Name,Price
0,F001,P3,8,ยางลบ,3
1,F002,P1,15,ปากกา,10
2,J001,P1,10,ปากกา,10
3,J002,P2,5,ดินสอ,5
4,J003,P1,20,ปากกา,10


แบบยาก (Pro Mode: Approximate Match / merge_asof) อันนี้ทีเด็ด
สถานการณ์จริง: ป๋ามีพัสดุหนัก 2.5 กิโล ในตารางเรทราคามีแค่ 1, 5, 10 กิโล ไม่มีเลข 2.5 เป๊ะๆ

VLOOKUP/Merge ธรรมดาจะหาไม่เจอ (ตาย)

XLOOKUP (Approximate) จะหาตัวที่ใกล้เคียงให้

ใน Pandas ใช้ pd.merge_asof (ต้องเรียงข้อมูลก่อนเสมอ)

In [12]:
# สมมติมีข้อมูลพัสดุที่ต้องการคิดเงิน
parcels = pd.DataFrame({'Parcel_ID': ['A', 'B'], 'Weight': [2.5, 9.8]})
rates = pd.read_csv('../datasets/multiple_data/shipping_Cost.csv')
rates['Max_Weight'] = rates['Max_Weight'].astype(float)

# 1. กฎเหล็ก: ข้อมูลต้องเรียงลำดับ (Sort) ก่อนเสมอ ไม่งั้นพัง
parcels = parcels.sort_values('Weight')
rates = rates.sort_values('Max_Weight')

# 2. ใช้ merge_asof (direction='forward' คือหาค่าถัดไปที่ครอบคลุมน้ำหนักเรา)
# เช่น หนัก 2.5 -> ไปตกที่เรท Max_Weight 5.0

cost_df = pd.merge_asof(
    parcels,
    rates,
    left_on='Weight',
    right_on='Max_Weight',
    direction='forward' # forward = หาค่าที่มากกว่าหรือเท่ากับตัวมันที่ใกล้ที่สุด
)

print("--- ผลลัพธ์ XLOOKUP (Approximate Match) ---")
print(cost_df)
print(parcels['Weight'].dtype)     # ดูว่าฝั่งซ้ายเป็นอะไร
print(rates['Max_Weight'].dtype)   # ดูว่าฝั่งขวาเป็นอะไร

--- ผลลัพธ์ XLOOKUP (Approximate Match) ---
  Parcel_ID  Weight  Max_Weight  Shipping_Cost
0         A     2.5         5.0             50
1         B     9.8        10.0             80
float64
float64


ผลลัพธ์: พัสดุหนัก 2.5 จะไปจับคู่กับ Max_Weight 5 ได้ค่าส่ง 50 บาท (ถูกต้องตามเรทการขนส่ง)

สรุปเทคนิควันนี้:

รวมไฟล์: ใช้ glob วนลูป + concat (งานซ้ำๆ เสร็จในวิเดียว)

VLOOKUP: ใช้ merge ระบุ left_on/right_on เมื่อชื่อคอลัมน์ไม่ตรงกัน

XLOOKUP (ช่วงข้อมูล): ใช้ merge_asof สำหรับข้อมูลที่เป็น Range (ราคา, เวลา, น้ำหนัก)

ป๋าลองก๊อปไปรันดูนะครับ ติดตรงไหนถามจุกเงินได้เลย! บทต่อไปเราจะไปเรื่อง "การจัดการข้อมูลวันที่และเวลา (Time Series)" หรือ "ทำ Pivot Table ขั้นสูง" ดีครับ?